In [80]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd()) # dosya yolunu ver
%run ./Model.ipynb
%run ../Dataset.ipynb

plt.rcParams['font.size'] = 20
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

/arf/home/tunal/ondemand/PhD Thesis Starting/01_SON/Tik-4/Tez/03-Spiral/02-UAE_for_Spiral-Copy1-Copy1


In [81]:
class Trainer:
    def __init__(self, model, optimizer, device='cpu', max_patience=20):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.device = device
        
        self.max_patience = max_patience
        self.best_val_loss = float('inf')
        self.patience = 0

        self.val_cost = []
        self.train_cost = []
    
    def train(self, train_loader, val_loader=None, epochs=10, print_every=1, name=None):
        self.model.train()
        
        for epoch in range(1, epochs + 1):
            total_loss = 0.0
            for x, x_, y in train_loader:  # label kullanılmıyor
                x = x.to(self.device)
                x_ = x_.to(self.device)

                with torch.no_grad():
                    z_hat_ = self.model.encoder(x_)
                z_hat, x_hat = self.model(x)
                
                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x.reshape(-1,3))

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            # Validation (opsiyonel)
            if val_loader:
                val_loss = self.validate(val_loader)
                
                # Early stopping kontrolü
                if val_loss < self.best_val_loss:
                    print('saved!')
                    torch.save(self.model, name + '.model')
                    self.best_val_loss = val_loss
                    self.patience = 0
        
                else:
                    self.patience = self.patience + 1
        
                if self.patience > self.max_patience:
                    break
                    
            if epoch % print_every == 0:
                print(f"Epoch {epoch:3d} | Train Loss: {total_loss / len(train_loader):.6f} | Validation Loss: {val_loss:.6f}")

            self.val_cost.append(val_loss)
            self.train_cost.append(total_loss / len(train_loader))
            
    def validate(self, val_loader):
        self.model.eval()
        total_loss = 0.0
        
        with torch.no_grad():
            for x, x_, _ in val_loader:
                x = x.to(self.device)
                x_ = x_.to(self.device)
                
                z_hat_, x_hat_ = self.model(x_)
                z_hat, x_hat = self.model(x)
                
                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x.reshape(-1,3))
                
                total_loss += loss.item()
                
        avg_loss = total_loss / len(val_loader)
        #print(f"→ Validation Loss: {avg_loss:.6f}")
        self.model.train()
        return avg_loss

In [82]:
# Custom Transform
class NoiseTransform:
    """Add some noise."""

    def __init__(self, split_ratio=0.001, dim=3):

        self.normal_dist = split_ratio*np.random.randn(dim,)
        
    def __call__(self, x):
      return x + self.normal_dist

In [83]:
# Hyper-Parameters & Settings

dataset_size = 2000
batch_size = 32
lr = 0.001

epochs = 1000
max_patience = 1000

split_ratio = 0.0001

In [84]:
# Dataset
train_dataset = SpiralDataset(mode='train', n_samples=dataset_size, transform=NoiseTransform(split_ratio))
val_dataset = SpiralDataset(mode='val', n_samples=dataset_size, transform=NoiseTransform(split_ratio))
test_dataset = SpiralDataset(mode='test', n_samples=dataset_size)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)

In [85]:
# Model
name = 'results/UAE_Spiral'
model = To_Uniform(
                 encoder_layers=[3, 128, 128, 128, 2],
                 decoder_layers=[2, 128, 128, 128, 3],
                 encoder_act=nn.ReLU,
                 decoder_act=nn.ReLU,
                 final_encoder_act=nn.Sigmoid,
                 final_decoder_act=nn.Sigmoid,
                 use_batchnorm=True
)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [86]:
# Training
trainer = Trainer(model, optimizer, device='cuda' if torch.cuda.is_available() else 'cpu', max_patience=max_patience)
trainer.train(train_loader, val_loader, epochs=epochs, name=name)

saved!
Epoch   1 | Train Loss: 0.012885 | Validation Loss: 0.005378
saved!
Epoch   2 | Train Loss: 0.004595 | Validation Loss: 0.003818
saved!
Epoch   3 | Train Loss: 0.003359 | Validation Loss: 0.001563
saved!
Epoch   4 | Train Loss: 0.002046 | Validation Loss: 0.001093
saved!
Epoch   5 | Train Loss: 0.001565 | Validation Loss: 0.000959
saved!
Epoch   6 | Train Loss: 0.001519 | Validation Loss: 0.000641
Epoch   7 | Train Loss: 0.001352 | Validation Loss: 0.000698
saved!
Epoch   8 | Train Loss: 0.001188 | Validation Loss: 0.000530
Epoch   9 | Train Loss: 0.001112 | Validation Loss: 0.000594
saved!
Epoch  10 | Train Loss: 0.001451 | Validation Loss: 0.000496
Epoch  11 | Train Loss: 0.001176 | Validation Loss: 0.000515
Epoch  12 | Train Loss: 0.001137 | Validation Loss: 0.000672
Epoch  13 | Train Loss: 0.001394 | Validation Loss: 0.000607
saved!
Epoch  14 | Train Loss: 0.001053 | Validation Loss: 0.000441
Epoch  15 | Train Loss: 0.001329 | Validation Loss: 0.000598
Epoch  16 | Train Loss

In [87]:
# CSV dosyasına kaydet
train_losses = trainer.train_cost  # liste veya numpy array
val_losses = trainer.val_cost

np.savetxt("results/losses.csv", 
           np.column_stack((train_losses, val_losses)),
           delimiter=",", 
           header="train_loss,val_loss", 
           comments="")